In [1]:
import pandas as pd
from pandas import read_csv
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV

In [2]:
# load data
df = pd.read_csv('./Data_income/clean_data',index_col=0)
df.head(3)

,age,workclass,fnlwgt,education-num,occupation,race,sex,capital-gain,capital-loss,hours-per-week,...,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife
0,39,5,77516.0,13.0,7,4,1,2174.0,0.0,40.0,...,0,1,0,0,0,1,0,0,0,0
1,50,6,83311.0,13.0,15,4,1,0.0,0.0,13.0,...,0,0,0,0,1,0,0,0,0,0
2,38,4,215646.0,9.0,3,4,1,0.0,0.0,40.0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education-num', 'occupation', 'race',
       'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
       'native-country', 'Y', 'marital-status_ Divorced',
       'marital-status_ Married-AF-spouse',
       'marital-status_ Married-civ-spouse',
       'marital-status_ Married-spouse-absent',
       'marital-status_ Never-married', 'marital-status_ Separated',
       'marital-status_ Widowed', 'relationship_ Husband',
       'relationship_ Not-in-family', 'relationship_ Other-relative',
       'relationship_ Own-child', 'relationship_ Unmarried',
       'relationship_ Wife'],
      dtype='object')

In [4]:
corr_matrix = df.corr()
print(corr_matrix["Y"].sort_values(ascending=False))

Y                                        1.000000
marital-status_ Married-civ-spouse       0.445853
relationship_ Husband                    0.403791
occupation                               0.344212
education-num                            0.332613
age                                      0.230369
hours-per-week                           0.227687
capital-gain                             0.223013
sex                                      0.214628
workclass                                0.170452
capital-loss                             0.147554
relationship_ Wife                       0.120484
race                                     0.096204
native-country                           0.094950
marital-status_ Married-AF-spouse        0.008977
fnlwgt                                  -0.006339
marital-status_ Married-spouse-absent   -0.039303
marital-status_ Widowed                 -0.065050
marital-status_ Separated               -0.073583
relationship_ Other-relative            -0.085601


In [5]:
lst1 = list(df.columns)
lst1.remove('Y')
lst11 = lst1[0:14]

# Separate out the target

In [6]:
X = df[lst1]
y = df['Y']
X.head(3)

,age,workclass,fnlwgt,education-num,occupation,race,sex,capital-gain,capital-loss,hours-per-week,...,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife
0,39,5,77516.0,13.0,7,4,1,2174.0,0.0,40.0,...,0,1,0,0,0,1,0,0,0,0
1,50,6,83311.0,13.0,15,4,1,0.0,0.0,13.0,...,0,0,0,0,1,0,0,0,0,0
2,38,4,215646.0,9.0,3,4,1,0.0,0.0,40.0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
y.shape, X.shape

((48842,), (48842, 24))

In [9]:
# Import module to split dataset
from sklearn.model_selection import train_test_split
# Split data set into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=50, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_rf))

0.8608793571172647


In [11]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(classification_report(y_test,y_pred_rf))



              precision    recall  f1-score   support

           0       0.87      0.96      0.91     14793
           1       0.81      0.56      0.66      4744

    accuracy                           0.86     19537
   macro avg       0.84      0.76      0.79     19537
weighted avg       0.86      0.86      0.85     19537



In [12]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_rf).ravel()
print([tp,fp])
print([fn,tn])

[2665, 639]
[2079, 14154]


In [13]:
error = (fp+fn)/(fp+fn+tp+tn)
error

0.13912064288273532

# Adaboost

In [14]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=200,learning_rate=0.05, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=2,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

In [15]:
y_pred = ada_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8670215488560168


In [15]:
# Import module to split dataset
from sklearn.model_selection import train_test_split
# Split data set into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [17]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

X.shape

              


(48842, 24)

In [18]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
clf.feature_importances_  


array([0.14340058, 0.03607667, 0.17258595, 0.10980317, 0.09802259,
       0.01421002, 0.01149735, 0.0802208 , 0.02537253, 0.0903701 ,
       0.02265341, 0.00846336, 0.00022324, 0.06211559, 0.00065665,
       0.03592936, 0.00124582, 0.0013737 , 0.05809001, 0.00610771,
       0.00135233, 0.00471531, 0.00321365, 0.01230008])

In [19]:

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new.shape 

(48842, 8)

In [22]:
# Import module to split dataset
from sklearn.model_selection import train_test_split
# Split data set into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3)

In [23]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=200,learning_rate=0.05, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=2,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

In [24]:
y_pred = ada_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8594144543779431


In [16]:
X.shape

(48842, 24)

In [17]:
X_test.shape, X_train.shape

((19537, 24), (29305, 24))

In [18]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
# X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:34200], X[34200:]
y_train, y_test = y[:34200], y[34200:]



clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=2, min_samples_leaf=5, random_state=42).fit(X_train, y_train)
clf.score(X_test, y_test)


0.8742658106816009

In [19]:
gb_clf2 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=2, random_state=42)
gb_clf2.fit(X_train, y_train)
predictions = gb_clf2.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix:
[[10487   709]
 [ 1191  2255]]
Classification Report
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     11196
           1       0.76      0.65      0.70      3446

    accuracy                           0.87     14642
   macro avg       0.83      0.80      0.81     14642
weighted avg       0.87      0.87      0.87     14642



In [20]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=150, learning_rate=learning_rate, max_depth=3, random_state=42)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))

Learning rate:  0.05
Accuracy score (training): 0.869
Accuracy score (validation): 0.870
Learning rate:  0.075
Accuracy score (training): 0.871
Accuracy score (validation): 0.871
Learning rate:  0.1
Accuracy score (training): 0.874
Accuracy score (validation): 0.872
Learning rate:  0.25
Accuracy score (training): 0.883
Accuracy score (validation): 0.876
Learning rate:  0.5
Accuracy score (training): 0.892
Accuracy score (validation): 0.872
Learning rate:  0.75
Accuracy score (training): 0.893
Accuracy score (validation): 0.869
Learning rate:  1
Accuracy score (training): 0.892
Accuracy score (validation): 0.867


In [21]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=150, min_samples_leaf=5, learning_rate=learning_rate, max_depth=3, random_state=42)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))

Learning rate:  0.05
Accuracy score (training): 0.869
Accuracy score (validation): 0.870
Learning rate:  0.075
Accuracy score (training): 0.872
Accuracy score (validation): 0.871
Learning rate:  0.1
Accuracy score (training): 0.874
Accuracy score (validation): 0.873
Learning rate:  0.25
Accuracy score (training): 0.883
Accuracy score (validation): 0.875
Learning rate:  0.5
Accuracy score (training): 0.890
Accuracy score (validation): 0.874
Learning rate:  0.75
Accuracy score (training): 0.894
Accuracy score (validation): 0.869
Learning rate:  1
Accuracy score (training): 0.893
Accuracy score (validation): 0.867


In [29]:
gb_clf2 = GradientBoostingClassifier(n_estimators=150, min_samples_leaf=5, learning_rate=0.25, max_depth=3, random_state=42)
gb_clf2.fit(X_train, y_train)
predictions = gb_clf2.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix:
[[10552   644]
 [ 1187  2259]]
Classification Report
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     11196
           1       0.78      0.66      0.71      3446

    accuracy                           0.87     14642
   macro avg       0.84      0.80      0.82     14642
weighted avg       0.87      0.87      0.87     14642



In [30]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print([tp,fp])
print([fn,tn])

[2259, 644]
[1187, 10552]


In [31]:
error = (fp+fn)/(fp+fn+tp+tn)
error

0.12505122251058598

In [25]:
# from sklearn.metrics import roc_curve, auc
# from sklearn import datasets
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import LinearSVC
# from sklearn.preprocessing import label_binarize
# import matplotlib.pyplot as plt

# # iris = datasets.load_iris()
# # X, y = iris.data, iris.target

# # y = label_binarize(y, classes=[0,1,2])
# n_classes = 2

# # # shuffle and split training and test sets
# # X_train, X_test, y_train, y_test =\
# #     train_test_split(X, y, test_size=0.33, random_state=0)

# # classifier
# clf = OneVsRestClassifier(LinearSVC(random_state=0))
# y_score = clf.fit(X_train, y_train).decision_function(X_test)

# # Compute ROC curve and ROC area for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# for i in range(n_classes):
#     fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
#     roc_auc[i] = auc(fpr[i], tpr[i])

# # Plot of a ROC curve for a specific class
# for i in range(n_classes):
#     plt.figure()
#     plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
#     plt.plot([0, 1], [0, 1], 'k--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver operating characteristic example')
#     plt.legend(loc="lower right")
#     plt.show()